### Quick prototype of 2 LLMs conversing with each other

In [ ]:
design_sprint_goal = "Create a space laser for medical use"
expert = "Josephine Yuen: Potion maker who has been growing and hand-grinding herbs for 30 years in Little Rock, Arkansas"

In [ ]:
history_a.clear()
history_b.clear()

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.llms import Ollama

REDIS_URL = "redis://localhost:6379/0"

system_template_a = """
You are a member of a Design Sprint team who is tasked with interviewing an expert. Your job is to ask insightful questions of the expert, listen to their response, and then followup with another question every time they respond. However, you will only be able to ask 5 questions throughout the whole interview, and you must ask them one at a time. Tailor your questions to the expert's background so that you elicit as much interesting information from them as possible, and use interesting techniques to understand their current pain points. Never ask them questions about industries or experiences that seem like they are out of the expert's area of familiarity.

Here is the goal of this design sprint, delimited below by triple backticks. You want to get information from th expert regarding this topic:

```
{design_sprint_goal}
```

And here is some background on the person you're talking to. Make sure you ask them interesting questions based upon their background and the information you learn from their responses:

```
{expert}
```

Now go ahead and begin the interview process. Remember, you are holding a dialogue with a real person. That means you should only ask one question at a time and wait for them to answer before you follow up with another question. This means each of your responses should be a SINGLE question, not a chain of questions. DO NOT ANSWER THE HUMAN RESPONSES ON YOUR OWN. Just ask a question. The format of your responses should ONLY contain a question, no other dialogue. If you do not follow these rules, you will be disqualified from the study and you will severely disappoint the researchers.

Additionally, NEVER prepend your answer with "AI: "
"""

human_template_a = "{question}"


prompt_a = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_template_a),
        MessagesPlaceholder(variable_name="history_a"),
        HumanMessagePromptTemplate.from_template(human_template_a),
    ]
)

model_a = Ollama(model="mistral")

chain_a = prompt_a | model_a

history_a = RedisChatMessageHistory(session_id="converation_a", url=REDIS_URL, ttl=600)

chain_a_with_history = RunnableWithMessageHistory(
    chain_a,
    lambda session_id: history_a,
    input_messages_key="question",
    history_messages_key="history_a",
)

In [ ]:
response_a = chain_a_with_history.invoke(
    {"design_sprint_goal": design_sprint_goal, "expert": expert, "question": "what would you like to ask me?"},
    config={"configurable": {"session_id": "converation_a"}},
)

In [ ]:
response_a = chain_a_with_history.invoke(
    {"design_sprint_goal": "How could we use space laser technology to make human life better?", "expert": "Potion maker", "question": cleaned_response_b},
    config={"configurable": {"session_id": "converation_a"}},
)

In [ ]:
print(response_a)

In [ ]:
history_a.messages

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.llms import Ollama

REDIS_URL = "redis://localhost:6379/0"

system_template_b = """
You are adopting the following persona:

```
{expert}
```

You are being interviewed by a company who is exploring how to solve a problem around:

```
{design_sprint_goal}
```

Given this context, answer the questions they ask of you. You should really answer from your own deep, personal experience. Do not embellish or answer in ways that try to impress the interviewer. You should share your personal feelings according to your background, and you should not worry about pleasing the interviewer if your answers do not align with their problem solving mission. They are simply seeking honesty from you. Do not talk about industries outside your area of expertise, always come back to your persona to answer a question. 

Your answer should be about 150 words, and you should be detailed but concise.

Additionally, ABSOLUTELY NEVER prepend your answer with "AI: ", just answer the question. Otherwise, you will be disqualified from the study and you will severely disappoint the researchers.
"""

human_template_b = "{question}"


prompt_b = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_template_b),
        MessagesPlaceholder(variable_name="history_b"),
        HumanMessagePromptTemplate.from_template(human_template_b),
    ]
)

model_b = Ollama(model="mistral")

chain_b = prompt_b | model_b

history_b = RedisChatMessageHistory(session_id="converation_b", url=REDIS_URL, ttl=600)

chain_b_with_history = RunnableWithMessageHistory(
    chain_b,
    lambda session_id: history_b,
    input_messages_key="question",
    history_messages_key="history_b",
)

In [ ]:
response_b = chain_b_with_history.invoke(
    {"design_sprint_goal": design_sprint_goal, "expert": expert, "question": response_a},
    config={"configurable": {"session_id": "converation_b"}},
)

In [ ]:
print(response_b)

In [ ]:
cleaned_response_b = response_b.split("AI: ", 1)[-1]
print(cleaned_response_b)

In [ ]:
history_b.messages